In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os
import sys
import time

In [ ]:
sys.path.append("/home/caleml/main-pe/")

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import Model, Input, Sequential
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D, Convolution2D, Activation, BatchNormalization, Reshape
from tensorflow.keras.layers import Permute, add, concatenate
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.applications import ResNet50

In [ ]:
from data.datasets.mpii import MpiiSinglePerson
from data.datasets.h36m import Human36M
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE
from data.loader import BatchLoader

from model import blocks
from model import layers
from model import losses
from model import config
from model import callbacks
from model.utils import pose_format

# model

In [ ]:
from model.networks.appearance_model import AppearanceModel
from model.networks.pose_model import PoseModel
from model.networks.multi_branch_model import MultiBranchModel

# all models are now in their respective file

In [ ]:
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import SeparableConv2D
from model.utils import math

def lin_interpolation_2d(inp, dim):
    num_rows, num_cols, num_filters = inp.get_shape().as_list()[1:]
    conv = SeparableConv2D(num_filters, (num_rows, num_cols), use_bias=False)
    x = conv(inp)

    w = conv.get_weights()
    w[0].fill(0)
    w[1].fill(0)
    linspace = math.linspace_2d(num_rows, num_cols, dim=dim)

    for i in range(num_filters):
        w[0][:,:, i, 0] = linspace[:,:]
        w[1][0, 0, i, i] = 1.

    conv.set_weights(w)
    conv.trainable = False
    
    x = Lambda(lambda x: tf.squeeze(x, axis=1))(x)
    x = Lambda(lambda x: tf.squeeze(x, axis=1))(x)
    x = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x)

    return x

# dataset

In [ ]:
h36m_path = "/share/DEEPLEARNING/datasets/h36m"
mpii_path = "/share/DEEPLEARNING/datasets/mpii"

In [ ]:
# h36m dataset loading
h36m = Human36M(h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames')

data_tr_h36m = BatchLoader(
    h36m, 
    ['frame'], 
    ['frame', 'pose', 'pose', 'pose', 'pose'],
    TRAIN_MODE, 
    batch_size=8,
    shuffle=True)

# batch_size=[batch_size_mpii, batch_size_mpii, batch_size_ar, batch_size_ar], 

In [ ]:
len(data_tr_h36m)

In [ ]:
a = data_tr_h36m.get_data(1, TRAIN_MODE)
print(type(a), a.keys())
print(a['pose'])
print("pose shape %s" % (str(a['pose'].shape)))
print("frame shape %s" % (str(a['frame'].shape)))

In [ ]:
# validation
h36m_val = BatchLoader(
    h36m, 
    ['frame'],
    ['pose_w', 'pose_uvd', 'afmat', 'camera', 'action'], 
    VALID_MODE,
    batch_size=h36m.get_length(VALID_MODE), 
    shuffle=True)

[x_val], [pw_val, puvd_val, afmat_val, scam_val, action] = h36m_val[0]

h36m_callback = H36MEvalCallback(x_val, pw_val, afmat_val, puvd_val[:,0,2], scam_val, action, logdir=logdir)

In [ ]:
# mpii = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf, poselayout=pose_format.pa17j3d)
mpii = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)

In [ ]:
data_tr_mpii = BatchLoader(
    mpii, 
    ['frame'], 
    ['frame', 'pose', 'pose', 'pose', 'pose'], 
    TRAIN_MODE,
    batch_size=20,
    shuffle=False)

In [ ]:
len(data_tr_mpii)

In [ ]:
a = data_tr_mpii.get_data(1, TRAIN_MODE)
print(type(a), a.keys())
print(a['pose'])
print("pose shape %s" % (str(a['pose'].shape)))
print("frame shape %s" % (str(a['frame'].shape)))

In [ ]:
b = data_tr_mpii[1]
print(type(b), len(b))
print(type(b[0]), len(b[0]))
print(b[0][0].shape)
print(type(b[1]), len(b[1]))
print(b[1][0].shape, b[1][1].shape)

import matplotlib.pyplot as plt
img = b[0][0][0]
print(img.shape)
imgplot = plt.imshow(img)
plt.show()

# training

In [ ]:
model = Encoder()
model.build()

# steps_per_epoch = h36m.get_length(TRAIN_MODE) // batch_size_h36m
steps_per_epoch = mpii.get_length(TRAIN_MODE) // batch_size_mpii

In [ ]:
model.train(data_tr, steps_per_epoch)

In [ ]:
model = AppearanceModel()
model.build()


In [ ]:
model_name = 'appearance'
dataset_name = 'mpii'
model_folder = '/home/caleml/pe_experiments/exp_%s_%s_%s' % (model_name, dataset_name, datetime.datetime.now().strftime("%Y%m%d%H%M")) 
os.makedirs(model_folder)
model.train(data_tr_mpii, steps_per_epoch=len(data_tr_mpii), model_folder=model_folder, n_epochs=60)

## Multib mpii

In [ ]:
model = MultiBranchModel(dim=2, n_joints=16, nb_pose_blocks=4)
model.build()

In [ ]:
model_name = 'multib'
dataset_name = 'mpii'
model_folder = '/home/caleml/pe_experiments/exp_%s_%s_%s' % (model_name, dataset_name, datetime.datetime.now().strftime("%Y%m%d%H%M")) 
os.makedirs(model_folder)
model.train(data_tr_mpii, steps_per_epoch=len(data_tr_mpii), model_folder=model_folder, n_epochs=60)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(os.environ["CUDA_VISIBLE_DEVICES"])

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

## Multib h36m

In [ ]:
model = MultiBranchModel(dim=3, n_joints=17, nb_pose_blocks=4)
model.build()

In [ ]:
model_name = 'separate'
dataset_name = 'h36m'
model_folder = '/home/caleml/pe_experiments/exp_%s_%s_%s' % (model_name, dataset_name, datetime.datetime.now().strftime("%Y%m%d%H%M")) 
os.makedirs(model_folder)
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=60)

## Baseline H36m

In [ ]:
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['pose', 'pose', 'pose', 'pose'],
        TRAIN_MODE, 
        batch_size=32,
        shuffle=True)

In [ ]:
model = MultiBranchModel(dim=3, n_joints=17, nb_pose_blocks=8)
model.build_pose_only()

In [ ]:
model_name = 'baseline'
dataset_name = 'h36m'
model_folder = '/home/caleml/pe_experiments/exp_%s_%s_%s' % (model_name, dataset_name, datetime.datetime.now().strftime("%Y%m%d%H%M")) 
os.makedirs(model_folder)
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=60)

# eval

## Appearance model

In [ ]:
model_folder = '/home/caleml/pe_experiments/exp_appearance_mpii_201902051901'
model_checkpoint = '/home/caleml/pe_experiments/exp_appearance_mpii_201902051901/weights_mpii_058.h5'  # weights
checkpoint_2 = '/home/caleml/pe_experiments/exp_appearance_mpii_201902061614/weights_mpii_013.h5'  # made with save_model

model = AppearanceModel()
model.load(checkpoint_2)

## Separate model

In [ ]:
# separate model

from model.activations import channel_softmax_2d
from model.losses import reconstruction_loss, pose_loss

checkpoint = "/home/caleml/pe_experiments/exp_separate_mpii_201903081936/appearance_mpii_060.h5"

custom_objects = {
    '_channel_softmax_2d': channel_softmax_2d(),
    '_rec_loss': reconstruction_loss(),
    '_pose_loss': pose_loss()
}

In [ ]:
eval_model = MultiBranchModel()
eval_model.load(checkpoint, custom_objects=custom_objects)

In [ ]:
mpii_eval = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)
data_val_mpii = BatchLoader(
    mpii_eval, 
    ['frame'], 
    ['frame', 'pose', 'afmat', 'headsize'], 
    mode=VALID_MODE,
    shuffle=False)

len(data_val_mpii)

In [ ]:
def eval_mpii_pckh(model_class, x_val, pose_val, headsize_val, refp=0.5):
    '''
    assumes one prediction per block
    '''
    model = model_class.model
    input_shape = model.inputs[0].shape
    num_blocks = len(model.outputs) - 1  # -1 because of imag reconstruction output
    
    print("eval input shape %s, num blocks %s" % (str(input_shape), num_blocks))
    
    print("eval data x: %s y: %s %s" % (x_val.shape, pose_val.shape, headsize_val.shape))
        
    pred = model.predict(x_val)
    print(pred[1].shape)
    
    scores = list()
    print(pose_val[0])
    print(pred[1][0])
        
    for i_block in range(num_blocks):
        y_pred = pred[i_block + 1]
        s = pckh(pose_val, y_pred, headsize_val, refp=refp)
        scores.append(s)
        
        pckh_per_joint(pose_val, y_pred, headsize_val, pose_format.pa16j2d, verbose=1)
        
    print(scores)
    
    

In [ ]:
def pckh(y_true, y_pred, head_size, refp=0.5):
    '''
    Compute the PCKh measure (using refp of the head size) on predicted samples
    
    y_true: [batch_size, nb_joints, 2]
    y_pred: [batch_size, nb_joints, 2]
    head_size: [batch_size, 1]
    '''

    assert y_true.shape == y_pred.shape
    assert len(y_true) == len(head_size)
    num_samples = len(y_true)

    # Ignore the joints 6 and 7 (pelvis and thorax respectively), according to the file 'annolist2matrix.m' WHY
    used_joints = [2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 8, 9]
    y_true = y_true[:, used_joints, :]
    y_pred = y_pred[:, used_joints, :]
    dist = np.zeros((num_samples, len(used_joints)))
    valid = np.zeros((num_samples, len(used_joints)))

    for i in range(num_samples):
        valid[i,:] = _valid_joints(y_true[i])
        
        norm = _norm(y_true[i] - y_pred[i], axis=1)
        dist[i,:] = _norm(y_true[i] - y_pred[i], axis=1) / head_size[i]
        print("distance norm between true %s and pred %s : %s (head size %s)" % (str(y_true[i]), str(y_pred[i]), norm, head_size[i]))
    match = (dist <= refp) * valid

    return match.sum() / valid.sum()


def pckh_per_joint(y_true, y_pred, head_size, pose_layout, refp=0.5, verbose=1):
    '''
    Compute the PCKh measure (using refp of the head size) on predicted
    samples per joint and output the results.

    y_true: [num_samples, nb_joints, 2]
    y_pred: [num_samples, nb_joints, 2]
    head_size: [num_samples, 1]
    pose_layout: from deephar.utils.pose
    '''

    assert y_true.shape == y_pred.shape
    assert len(y_true) == len(head_size)

    num_samples = len(y_true)
    num_joints = pose_layout.num_joints
    dist = np.zeros((num_samples, num_joints))
    valid = np.zeros((num_samples, num_joints))

    for i in range(num_samples):
        valid[i,:] = _valid_joints(y_true[i])
        dist[i,:] = _norm(y_true[i] - y_pred[i], axis=1) / head_size[i]

    for j in range(num_joints):
        jname = pose_layout.joint_names[j]
        space = 7*' '
        ss = len(space) - len(jname)
        if verbose:
            printc(HEADER, jname + space[0:ss] + '| ')
    if verbose:
        print ('')

    match = (dist <= refp) * valid
    for j in range(num_joints):
        pck = match[:, j].sum() / valid[:, j].sum()
        if verbose:
            printc(OKBLUE, ' %.2f | ' % (100 * pck))
    if verbose:
        print ('')

In [ ]:
import sys

HEADER = '\033[95m'
OKBLUE = '\033[94m'
OKGREEN = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
ENDC = '\033[0m'

def printc(color, vmsg):
    print (color + vmsg + ENDC, end='')
    sys.stdout.flush()
    
def _valid_joints(y, min_valid=-1e6):
    def and_all(x):
        if x.all():
            return 1
        return 0

    return np.apply_along_axis(and_all, axis=1, arr=(y > min_valid))

def _norm(x, axis=None):
    return np.sqrt(np.sum(np.power(x, 2), axis=axis))

In [ ]:
[x_val], [y_val, pose_val, afmat_val, head_val] = data_val_mpii[0]
eval_mpii_pckh(eval_model, x_val, pose_val, head_val)

In [ ]:
data = data_val_mpii.get_data(1, VALID_MODE)
print(data['frame'].shape)
pred = eval_model.predict(data['frame'])


## Viz

In [ ]:
import matplotlib.pyplot as plt

n_cols = 2
n_rows = len(data['frame'])
fig = plt.figure(figsize=(8, 75))

print(len(data['frame']))
print(len(pred[0]))

i_img = 1
for i, data_img in enumerate(data['frame']):
    fig.add_subplot(n_rows, n_cols, i_img)
    plt.imshow(data_img)
    
    pred_img = pred[0][i]
    fig.add_subplot(n_rows, n_cols, i_img + 1)
    plt.imshow(pred_img)
    
    i_img += 2
    
plt.show()


In [ ]:
img = pred[0][0]
print(img.shape)
imgplot = plt.imshow(img)
plt.show()

data_img = data['frame'][0]
imgplot = plt.imshow(data_img)
plt.show()

# debug

In [ ]:
def pouet(definition):
    
    ret = list()
    for elt in definition:
        ret.append('truc')
        
    return tuple(ret)

a = pouet('a')
        
    

In [ ]:
a

In [ ]:
b

In [ ]:
b

In [ ]:
tf.max

In [ ]:
def wrapper(i):
    def loss(a):
        return a * i
    return loss

losses = list()
for i in range(10):
    losses.append(wrapper(i))
    
print(len(losses))
for loss_fn in losses:
    print(loss_fn(10))